# Fase 5 - Interface Grafica
Vamos agora pegar tudo que aprendemos e juntar em uma interface grafica utilizavel.
Transformando nossos codigos em funções a serem chamadas.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime
import webbrowser


### Metodos

#### Coleta de Dados.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Configurar as opções do Chrome para o modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Criar o driver do Chrome com as opções configuradas
driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe', options=chrome_options)

# Exemplo de uso do WebDriver em modo headless
driver.get("https://www.google.com")
print(driver.title)

# Fechar o WebDriver
driver.quit()


C:\Users\berna\AppData\Local\Temp\ipykernel_15216\1374954969.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe', options=chrome_options)


Google


In [3]:
from selenium.webdriver.chrome.options import Options

def pesquisar(Produto):
    produto = str(Produto).upper()
    listaX = []

    # Configurar as opções do Chrome para o modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    # Criar o driver do Chrome com as opções configuradas
    driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe', options=chrome_options)
    # Entrando no catalogo da magalu
    url = 'https://www.magazineluiza.com.br/busca/{}/'.format(produto)
    driver.get(url)
    #Raspando os elementos
    linha = 1
    html = '/html/body/div[1]/div/main/section[4]/div[3]/div/ul/'
    while True:
        try:
            html = '/html/body/div[1]/div/main/section[4]/div[3]/div/ul/'
            listaInterna = []
            #Coletando os valores da pagina
            try: # PRODUTO
                nome = driver.find_element(By.XPATH,html +'li[{}]/a/div[3]/h2'.format(linha)).text
            except:
                html = '/html/body/div[1]/div/main/section[4]/div[4]/div/ul/'
                nome = driver.find_element(By.XPATH,html +'li[{}]/a/div[3]/h2'.format(linha)).text
            try: # PREÇO
                price = driver.find_element(By.XPATH, html +'li[{}]/a/div[3]/div[2]/div/div/p'.format(linha)).text
            except:
                price = driver.find_element(By.XPATH, html +'li[{}]/a/div[3]/div[1]/div/div/p'.format(linha)).text
                #Link para compra    
            link = driver.find_element(By.XPATH,html +'li[{}]/a'.format(linha)).get_attribute('href')
            
            listaInterna.append(nome)
            listaInterna.append(price)
            listaInterna.append(link)
            listaX.append(listaInterna) 
            #print('Tudo Ok')
            linha = linha + 1      
            if linha == 20: break
        except:
            if linha == 1: print("Magualu is Fail")
            break
    #Convertendo Price to float
    y = 0
    for x in listaX:
        limpeza = x[1][3:].replace('.','')
        listaX[y][1] = float(limpeza.replace(',','.'))
        #print(listaX[y][1]) 
        y += 1
    #Menor
    produtoHoje = []
    for x in listaX:
        if produtoHoje == []: produtoHoje = x 
        if produtoHoje[1] > x[1]: produtoHoje = x 

    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    # Obter os componentes da data e hora

    produtoHoje.append(f"{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year}")
    return produtoHoje
    
        

#### Json stufs.

In [4]:
    # novo_objeto = json.dumps(menor)
    # Aqui abrimos o nosso arquivo para poder mecher nele
def salvajson( target = "logFase5.json", Produto = "Celular",produtoHoje = []):   
        produto = Produto     
        with open(target, 'r') as arquivo:
            dados = json.load(arquivo)
        # -----------------------------------------------------------------------------------

        if produto in dados: #Varrendo nossos dados para encontrar nosso produto:
            #Produto Lista existe agora vamos adicionar o produto de Hoje.
            validar = True # Aqui o validar vai ser positivo para adicionar o novo elemento a lista.
                #Se encontrado enquanto vasculhamos Validar sera neativado e não vamos repetir produtos.
            for sub_cell in dados[produto]: #Tentando verificar se o produto de hj existe
                if sub_cell[3] == produtoHoje[3]:
                    print('Repitido')
                    validar = False
                    break
            if validar: dados[produto].append(produtoHoje)
        else: # Não existe a lista do produto então a criamos
            dados[produto] = [produtoHoje]
            
        # Salvando
        with open("logFase5.json", "w") as arquivo:
            json.dump(dados, arquivo, indent=2)
        
        if False: #Só vai escrever
            with open('logFase5.json', 'w') as file:
            #dados.append(novo_objeto)
                json.dump(menor, file, indent=4)

        


In [5]:
    # Retornando o menor Valor
def consultarHistorico(produto):
        with open('logFase5.json', 'r') as arquivo:
            dados = json.load(arquivo)
        menor = []
        for x in dados[produto]:
            if menor == []: menor = x 
            if menor[1] > x[1]: menor = x
        print(menor)

In [6]:
def varredura_total(lista_elemento = 'logFase5.json'):
    with open(lista_elemento, 'r') as arquivo:
        dados = json.load(arquivo)
    lista_total = []

    for x1 in dados: #Estamos desfasendo a biblioteca para então podermso montar a lista grande.
        for x2 in dados[x1]:
            Pnome = x1
            nome = x2[0]
            price = x2[1]
            link = x2[2]
            data = x2[3]
            lista_total.append([Pnome,nome,price, link, data]) # Lista grande montada
    return lista_total

### Interface Grafica
Vamos la montar isso, vamos fazer simples, uma tela com algumas opções:

Uma barra para colocar o nome do produto.

Depois de Enviado ele retorna o produto com nome, preço e link

In [7]:
import PySimpleGUI as sg
import webbrowser

def janela_main():
    layout = [
        [sg.Text("Digite o nome do produto:")],
        [sg.Input(key="-NOME-")],
        [sg.Button("Buscar", key="Enviar"),[sg.Button("Visualizar Ultimo Produto", key="-visualizar_produto-", visible=False, disabled=True)]],
        [sg.Button("Historico", key="janela_Historico")]
    ]
    window = sg.Window("Caçador de Preços", layout,size=(400, 200))
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "Enviar": # Aqui vamos adicionar o chamado aos nossos metodos
            produtohoje = pesquisar(values["-NOME-"])
            produto = str(values["-NOME-"]).upper()
            window.hide()  
            janela_produto(produto, produtohoje)
            window["-visualizar_produto-"].update(visible=True, disabled=False)
            window.un_hide()

        if event == "-visualizar_produto-":
            window.hide()  
            janela_produto(produto, produtohoje)
            window.un_hide()

        if event== "janela_Historico":
            janela_Historico()

    window.close()
#Janela onde sera exibido os produtos
def janela_produto( produto, produtohoje):
    layout = [
        [sg.Text("Resultado da busca")],
        [sg.Text(f"Nome digitado: {produto} \nModelo encontrado:"+
            f"{produtohoje[0][:]}\nR${produtohoje[1]} \nLink: {produtohoje[2]}")        
        ],
        [sg.Button("Abrir Link"), sg.Button("Voltar")],
        [sg.Button("Salvar")]
    ]

    window = sg.Window("Busca Efetuada", layout,size=(400, 200))

    while True:
        event, _ = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "Voltar":
            window.close()  # Fecha a janela secundária e retorna para a janela principal
            break
        if event == "Abrir Link":
            link = produtohoje[2]
            webbrowser.open(link)
        if event == "Salvar":
            salvajson(Produto = produto, produtoHoje =produtohoje)

# Janela de Historico
def janela_Historico(lista_elemento = 'logFase5.json',elementos_por_pagina= 5 ):
    conjunto_elementos = varredura_total()
    total_elementos = len(conjunto_elementos)
    paginas = (total_elementos + elementos_por_pagina - 1) // elementos_por_pagina
    # Cálculo do total de páginas
    #Matematica para calcular o Tamanho da pagna certo
   # if elementos_por_pagina >= len(conjunto_elementos): 
   #     elementos_por_pagina = len(conjunto_elementos)

    #Layout
    layout_pagina = [
        [sg.Text(f"{i +1}- {conjunto_elementos[i][0]}: {conjunto_elementos[i][1][:80]} - R${conjunto_elementos[i][2]} ", key=f"-ELEMENTO-{i}-"),
         sg.Button("Entrar Link", key=f"-LINK-{i}-")] for i in range(elementos_por_pagina)
    ]
    layout_botoes = [
        [sg.Button("Anterior"), sg.Button("Próximo"), sg.Button("Fechar")]
    ]

    layout = layout_pagina + layout_botoes

    window = sg.Window("Lista de Elementos", layout, size=(800, 200))

    pagina_atual = 0

    while True:
        event, _ = window.read()
        if event == sg.WINDOW_CLOSED or event == "Fechar":
            break
        if event == "Próximo" and pagina_atual < paginas - 1:
            pagina_atual += 1
            for i in range(elementos_por_pagina):
                elemento_index = pagina_atual * elementos_por_pagina + i
                if elemento_index < total_elementos:
                    print(elemento_index)
                    window[f"-ELEMENTO-{i}-"].update(f"{elemento_index +1}- {conjunto_elementos[elemento_index][0]}: {conjunto_elementos[elemento_index][1][:80]} - R${conjunto_elementos[elemento_index][2]} ")
                else:
                    window[f"-ELEMENTO-{i}-"].update("VAZIO")
        if event == "Anterior" and pagina_atual > 0:
            pagina_atual -= 1
            for i in range(elementos_por_pagina):
                elemento_index = pagina_atual * elementos_por_pagina + i
                if elemento_index < total_elementos:
                    window[f"-ELEMENTO-{i}-"].update(f"{elemento_index +1}- {conjunto_elementos[elemento_index][0]}: {conjunto_elementos[elemento_index][1][:80]} - R${conjunto_elementos[elemento_index][2]} ")
                else:
                    window[f"-ELEMENTO-{i}-"].update("")
                    
                # Lidar com o evento de clique em um botão link
        for i in range(elementos_por_pagina):
            if event == f"-LINK-{i}-":
                link_index = pagina_atual * elementos_por_pagina + i
                if link_index < total_elementos:
                    # Aqui você pode abrir o link no navegador usando a biblioteca webbrowser
                    import webbrowser
                    webbrowser.open(conjunto_elementos[link_index][3])
    window.close()




C:\Users\berna\AppData\Local\Temp\ipykernel_15216\1098604138.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe', options=chrome_options)


In [8]:
# Varredura Total: 
''' vamos varrer nosso json voltando os dados dos elementos:
esse metodo deve retornar uma unica lista com os dados:
    (Produto, Nome, Preço, Link, Data)
    varrer totalmente nosso Json e coletar isso, usando bonecas russas e cascatas conseguimos.
    o Return é uma Lista


 '''



' vamos varrer nosso json voltando os dados dos elementos:\nesse metodo deve retornar uma unica lista com os dados:\n    (Produto, Nome, Preço, Link, Data)\n    varrer totalmente nosso Json e coletar isso, usando bonecas russas e cascatas conseguimos.\n    o Return é uma Lista\n\n\n '

Agora o que posso fazer para melhorar?

• Usar Json para ter uma lista de produtos salvos.
• A lista sera navegavel entre abas.
• ela ficara abaixo do menu principal, tendo a poção de consultar ou fazer uma nova porcura.
 ! Vamos fazer primeiro como uma janela a parte.

# MAIN


In [9]:


if True:
    janela_main()